In [19]:
import pprint


text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# 创建一个集合 用来存储一些介词
stoplist = set('for a of the and to in'.split(' '))

# print(type(stoplist))

# .lower() 将文本转换成小写字母 .split() 对document进行分词
# 最后通过if来排除常见的介词
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in text_corpus
]

# 计算单词出现的频率
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# 输出不止出现一次的单词
# 通过双重循环 将二维数组转换为一维数组
processed_corpus = set ([
   token for text in texts for token in text if frequency[token]> 1 
])

pprint.pprint(processed_corpus)
print('预处理语料库长度：',len(processed_corpus))

{'computer',
 'eps',
 'graph',
 'human',
 'interface',
 'minors',
 'response',
 'survey',
 'system',
 'time',
 'trees',
 'user'}
预处理语料库长度： 12


In [7]:
from gensim import corpora
    
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


In [20]:
pprint.pprint(dictionary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


In [25]:
new_doc = "Human computer interation"
# new_doc = "Human use computer, Human eps computer"

new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


在每个元组中，第一个数值表示单词的ID，第二个单词表示单词出现的次数

In [27]:
bow_corpus = [ dictionary.doc2bow(text.lower().split()) for text in text_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


## Model

In [32]:
from gensim import models

tfidf = models.TfidfModel(bow_corpus)

words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


每个元组中，第一个数值表示单词的ID，第二个表示tf-idf权重

In [36]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

query_document = 'system enginnering'.split()
query_bow = dictionary.doc2bow(query_document)
print('query_bow\n', query_bow)

sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

query_bow
 [(5, 1)]
[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


In [37]:
for document_number, score in sorted(enumerate(sims), key=lambda x:x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


In [40]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### From Strings to Vectors

In [16]:
import pprint
from collections import defaultdict

# 导入9句话
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

stoplist = set('for a of the and to in'.split(' '))
texts = [
    [word for word in document.lower().split() if word not in stoplist ] 
    for document in documents
]

frequency = defaultdict(int)
for text in texts:
    for word in text:
        # 使用方法相当于字典
        frequency[word] += 1
        
# 去除只出现一token次的单词
texts = [[token for token in text if frequency[token]>1 ] for text in texts]

from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('./tmp/deerwester.dict')
print(dictionary)

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


In [17]:
# 得到单词的ID
print(dictionary.token2id)

new_doc = "human computer iteraction"
new_vect = dictionary.doc2bow(new_doc.lower().split())
print('new_vect: ', '\n', new_vect)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}
new_vect:  
 [(0, 1), (1, 1)]


In [18]:
corpus = [dictionary.doc2bow(text) for text in texts]
# 保存上述数据
corpora.MmCorpus.serialize('./tmp/deerwester.mm', corpus)
pprint.pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


In [27]:
from smart_open import open

class MyCorpus:
    def __iter__(self):
        # 这里也可以通过http协议来访问网络上的文件
        for line in open('./tmp/mycorpus.txt'):
            # 通过迭代返回内容
            yield dictionary.doc2bow(line.lower().split())

In [28]:
corpus_memory_friendly = MyCorpus()
print(corpus_memory_friendly)

# corpus现在是个对象 
# 不能直接打印输出 要通过循环的方式遍历
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


虽然输出结果与上述相同，但是通过这样的方式，哦对内存是更加友好的。

In [40]:
# 将corpus中的所有单词构成一个字典
dictionary = corpora.Dictionary(line.lower().split() for line in open('./tmp/mycorpus.txt'))
# 获得常用符号的id
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
# 获得只出现一次的id
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items()
          if docfreq == 1]

# 将上述id进行过滤
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print(dictionary)


Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


In [44]:
corpus = [[(1, 0.5)], []]
# 保存一个Matrix Market文件
corpora.MmCorpus.serialize('./tmp/corpus.mm', corpus)

corpus = corpora.MmCorpus('./tmp/corpus.mm')
print(corpus)
print(list(corpus))

MmCorpus(2 documents, 2 features, 1 non-zero entries)
[[(1, 0.5)], []]


In [54]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5, 2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
print(numpy_matrix)

for doc in corpus:
    print(doc)

[[7 0]
 [6 3]
 [2 9]
 [1 8]
 [1 7]]
[(0, 7.0), (1, 6.0), (2, 2.0), (3, 1.0), (4, 1.0)]
[(1, 3.0), (2, 9.0), (3, 8.0), (4, 7.0)]


## Topics and Transformations

In [63]:
import pprint
from collections import defaultdict
from gensim import corpora

# 导入9句话
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

stoplist = set('for a of the and to in'.split(' '))
texts = [
    [word for word in document.lower().split() if word not in stoplist ] 
    for document in documents
]

frequency = defaultdict(int)
for text in texts:
    for word in text:
        # 使用方法相当于字典
        frequency[word] += 1
        
# 去除只出现一token次的单词
texts = [[token for token in text if frequency[token]>1 ] for text in texts]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

from gensim import models

tfidf = models.TfidfModel(corpus)

In [64]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


`tfidf`可以用于将任何矩阵从旧的表示方法（单词的出现次数-整形）转换为新的表示方法（tfidf权重-浮点型）

In [66]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [67]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]

通过Laten Semantic Indexing将Tf-Idf转换成一个latent 2-D空间。

In [68]:
lsi_model.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]